In [2]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories

# Importing the classes and functions from the geometry folder
from geometry.mesh import Mesh 
from geometry.utils import *

# Importing the classes and functions from the optimization folder
from optimization.Planarity import Planarity
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

# Load Meshes

## Read Meshes

In [7]:
# You can use either igl or Mesh to load meshes

# igl can only load triangular meshes, it return a tuple (V, F)
v, f = igl.read_triangle_mesh(dir_path+"/models/Hall.obj")

# If you use the self implemented Mesh class, you can load any type of mesh
mesh = Mesh() # Create an empty mesh
mesh.read_obj_file(dir_path+"/models/Hall.obj") # Load the mesh from the obj file

# igl only return v and f. However the Mesh class has implemented a Half Edge data structure
# More information: https://jerryyin.info/geometry-processing-algorithms/half-edge/
# You can check the folder geometry/mesh.py to see how the half edge data structure is implemented

# To acces the vertices and faces of the mesh you can use the following commands
vertices = mesh.vertices
faces = mesh.faces()

print(f"igl vertices:\n {v[:5]},\n igl faces:\n {f[:5]}")

print(f"Mesh vertices:\n {vertices[:5]},\n Mesh faces:\n {faces[:5]}")

Mesh Data Structure: |V| = 3330, |F| = 6293, |E| = 9622
igl vertices:
 [[ 8.147727  8.511167  5.435742]
 [ 8.48664   7.833783  5.361411]
 [ 7.799988  8.175888  4.931837]
 [32.59761  20.012056  5.150359]
 [32.977852 20.232679  4.539225]],
 igl faces:
 [[ 1  0  2]
 [ 4  3  5]
 [ 7  6  8]
 [10  9 11]
 [13 12 14]]
Mesh vertices:
 [[ 8.147727  8.511167  5.435742]
 [ 8.48664   7.833783  5.361411]
 [ 7.799988  8.175888  4.931837]
 [32.59761  20.012056  5.150359]
 [32.977852 20.232679  4.539225]],
 Mesh faces:
 [[ 1  0  2]
 [ 4  3  5]
 [ 7  6  8]
 [10  9 11]
 [13 12 14]]


## Create Meshes

You can create meshes by defining its vertices and faces list. 

In [8]:
# Random vertices
v = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 1],
    ])

# Random faces
f = np.array([
    [0, 1, 2],
    [0, 2, 3],
    ])

# Create a mesh from vertices and faces
mesh = Mesh()
mesh.make_mesh(v, f)

print(f"Mesh vertices:\n {mesh.vertices},\n Mesh faces:\n {mesh.faces()}")

Mesh Data Structure: |V| = 4, |F| = 2, |E| = 5
Mesh vertices:
 [[0. 0. 1.]
 [1. 0. 0.]
 [1. 1. 0.]
 [0. 1. 1.]],
 Mesh faces:
 [[0 1 2]
 [0 2 3]]


One of the advantages of the Halfedge data structure is that we can acces very easily to certain properties of the mesh. For example obtaining the faces neighbor to a vertex

In [10]:
# Neighbor faces to vertices
# Each row contain the neighbor faces index to the vertex with the same index
nf = mesh.vertex_ring_faces_list()

# Neighbor vertices to vertex 0 
print(nf[0])

[0, 1, -1]


The number -1 refers to a halfedge with no face or boundary halfedge. In general means that vertex 0 is a boundary vertex. 

# Visualization 

So far we have seen how to load meshes and create meshes but we haven't visualize them. Here I will show the alternatives for visualization either using **meshplot** or **vedo**. 

The adventage of **meshplot** is that is really easy to use and fast. Moreover, it is possible to visualize the evolution of an optimization process. However, it only work for triangular meshes and for ourporpuses is not always the case.

[**Vedo**](https://vedo.embl.es/docs/vedo.html) on the other hand allow to visualize any kind of polygonal meshes without restrictions, and also contain some mesh operations that can be helpfull from the geometric processing point of view. For example mesh intersection, boolean operations, group, etc. 

## Mesh plot

In [11]:
# Load mesh
# Random vertices
v = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 1],
    ])

# Random faces
f = np.array([
    [0, 1, 2],
    [0, 2, 3],
    ])

# Visualize mesh
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…

Polygonal shape problems. In the following example we will try to visualize a simple hexagon given by 6 vertices and only one face. 

In [20]:
# Create an hex mesh
# Hexagon center at the origin
v = np.array([
    [0, 0, 0], # 0
    [1, 0, 0], # 1
    [1.5, np.sqrt(3) / 2, 0], # 2
    [1, np.sqrt(3), 0], # 3
    [0, np.sqrt(3), 0], # 4
    [-0.5, np.sqrt(3) / 2, 0] # 5
])

    

# Faces
f = np.array([
    [0, 1, 2, 3, 4, 5]
    ])

# Visualize mesh
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.86…

We can see that meshplot interpret the face as two separated triangles instead of drawing the correct shape of the hexagon. We can do some extra work to triangulate the hexagon but it could be tedius and a waste of time.

## Vedo

In [17]:
# Load mesh
# Random vertices
v = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 1],
    ])

# Random faces
f = np.array([
    [0, 1, 2],
    [0, 2, 3],
    ])

# Create mesh vedo
mesh = vd.Mesh([v, f], c="red", alpha=0.5)

# Visualize wireframe
edges = mesh.clone().wireframe().c("black").lw(0.1)

# Visualize mesh
vd.show(mesh, edges)


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

As you can see to use vedo we need to define more things to obtain a nice visualization of the mesh in comparison with meshplot. However, we can use it to visualize any polygonal mesh. 

In [21]:
# Create an hex mesh
# Hexagon center at the origin
v = np.array([
    [0, 0, 0], # 0
    [1, 0, 0], # 1
    [1.5, np.sqrt(3) / 2, 0], # 2
    [1, np.sqrt(3), 0], # 3
    [0, np.sqrt(3), 0], # 4
    [-0.5, np.sqrt(3) / 2, 0] # 5
])

    

# Faces
f = np.array([
    [0, 1, 2, 3, 4, 5]
    ])

# Visualize mesh vedo
mesh = vd.Mesh([v, f], c="red", alpha=0.5)

# Visualize wireframe
edges = mesh.clone().wireframe().c("black").lw(0.1)

# Visualize mesh
vd.show(mesh, edges)


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

As you can see vedo triangulate the shape automatically and shows the correct shape. But let it try in a more complicated example made using Mesh class. 

In [13]:
# Load mesh
mesh = Mesh()
mesh.read_obj_file(dir_path+"/models/catenoid_def_1.obj")

# Faces 
f = mesh.faces()
# Vertices
v = mesh.vertices

# Compute barycenter of the mesh
bar = v[f].mean(axis=1)

# Get dual topology of the mesh
dual = mesh.dual_top()


#Visualize mesh vedo
mesh = vd.Mesh([v, f], c="red", alpha=0.5)

#Vis dual mesh vedo
dual_mesh = vd.Mesh([bar, dual], c="blue", alpha=0.5)

dual_mesh.lw(2.5).lc('white')

#Visualize mesh
vd.show(mesh, dual_mesh, __doc__, axes=11, viewup="z")

Mesh Data Structure: |V| = 336, |F| = 601, |E| = 936


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

It visualize the correct mesh but not show edges. 

## Polyscope

Other alternative  for visualization is polyscope, which is so far the one that work the best but is used only for visualization. 

In [14]:
# Initialize polyscope
ps.init()


### Register a mesh
# `verts` is a Nx3 numpy array of vertex positions
# `faces` is a Fx3 array of indices, or a nested list
ps.register_surface_mesh("Mesh", v, f, smooth_shade=True)
ps.register_surface_mesh("Dual", bar, dual, smooth_shade=True)

# Add a scalar function and a vector function defined on the mesh
# vertex_scalar is a length V numpy array of values
# face_vectors is an Fx3 array of vectors per face


# View the point cloud and mesh we just registered in the 3D UI
ps.show()


# Optimization

## Planarity

Here we are going to implement a simple minimization problem. We are goint to consider a mesh with four quad faces, and we want the faces to be planar. 
The mesh $M = \{ V, \ F, \ E \}$, where $V$ is the set of vertices, $F$ the set of faces and $E$ the set of edges. The planarity condition requires that per each face $f = v_i v_j v_k v_l$ the four vertices are coplanar. There are many geometric ways to impose this but the most efficient and easy is to add an auxiliary variable $n_f$ that represent a normal vector per ach face and we optimize per each face the energy:
$$  || n_f\cdot (v_j - v_i) ||^2; \quad \ \ v_i, \ v_f \in E(f).$$

Let's start by defining the initial mesh with not planar quads.

In [4]:
# Vertices
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
# Faces
fcs = np.array([[0, 1, 6, 3], 
                [2, 0, 3, 5], 
                [7, 4, 0, 2], 
                [4, 8, 1, 0]])

# Initialize polyscope
ps.init()


### Register a mesh
ps.register_surface_mesh("Init_Mesh", v, fcs, smooth_shade=True)

# View the point cloud and mesh we just registered in the 3D UI
ps.show()



[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 525.125.06


The full energy that we want to minimize is,
$$ E_{planarity} = \sum_{f\in F} \sum_{v_i, v_j \in E(f)} || n_f \cdot (v_j - v_i) ||^2 + \sum_{f \in F} || n_f \cdot n_f - 1 ||^2$$

The method that we are going to use the called [Levenberg-Marquart](https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm). For simplicity I am going to describe the main things we need to optimize the energy. 

The main idea is that our energy is of the form:
$$ E = \sum_{i}^n || \phi_i(X) ||^2, $$
where $X\in \mathbb{R}^m$  is a vector of variables and $\phi: \mathbb{R}^m \to \mathbb{R}$. Then, what the LM method does is to solve iteratively a linear system that will guide us to an optimal solution,
$$ (J^T J + \lambda \mathbb{I}) \delta_x =  - J^T\ r,$$
where
$$ J_{ij} = \frac{ \partial \phi_i{X} }{\partial x_j},$$
$$ r_i = \phi_i(X).$$
$\lambda$ is a parameter avoid the non solvability in general is a small value that in practice is computed as the maximum diagonal entry of $J^T J$ times $10^-6$.

## $J$ Computation

Let's see how we can compute $J$ for our example. We can see that $J$ will be a matrix where the rows are equivalent to each constraint or function $\phi_i$ and each column correspond to the derivatives of our variables. In our problem our variables are the vertices $V$ and the auxiliar normals $n_f$. Moreover, we have one constriant per each edge in each face. We can even rewrite the energy as,
$$ E_{planarity} = \sum_{f\in F} \sum_{i = 0}^3 || n_f \cdot (v_i - v_{i+1}) ||^2 + \sum_{f \in F} || n_f \cdot n_f - 1 ||^2$$
where the subscripts of the vertices are taken as $\mod 4$. This means that we have in total $4|F|$ per each edge in a quad plus $|F|$ in the second sum related to the normalization of the normal vectors as constaints, and $3|V|+3|F|$ variables. We consider $3|V|$ because we have three coordinates per each vertex and three coodinate per each normal vector $n_f$.

In [7]:
V = len(v) # Number of vertices
F = len(fcs) # Number of faces

# Init the J matrix
J = np.zeros((5*F, 3*V+3*F))

Now we need to create a function that take as input the information of the mesh $v$ and $f$ and put the corresponding values in the matrix $J$. Let's define the structure of $J$ as the first $3|V|$ columns related to the vertices derivatives and $F$ related to the normal derivatives. Fixing $i$ and $f$ we can see that the 
$$\partial_{x_i} ( n_f \cdot(x_i - x_{i+1}) ) = n_f,$$
$$\partial_{x_{i+1}} ( n_f \cdot(x_i - x_{i+1}) ) = - n_f,$$
$$\partial_{n_{f}} ( n_f \cdot(x_i - x_{i+1}) ) = (x_i - x_{i+1}).$$
$$\partial_{n_{f}} ( n_f \cdot n_f - 1) ) = n_f$$

Let's remember that $f$ can be the index of the face in the list of faces *fcs* and *i* means the vertex indices in *fcs[f]*.

**Remark:** In the previus part we define the derivative with respect to $\partial_{x_i}$ but $x_i$ is a vector which means that what we need to fill in $J$ is the derivative with respect to each coodinate, i.e., $(\partial_{x_i})_x ( n_f \cdot(x_i - x_{i+1}) ) = (n_f)_x $

In [14]:

# Implement a function to fill the J matrix
def compute_J(X,f):
    # Hint: consider the first 4 rows of the J matrix 
    # as the ones related to the first sum and the last rows related to the second sum
    
    pass



Now, that we have computed $J$ we can easily compute $r$.

In [15]:
# Implement a function tha retunr the residual vector
def compute_r(X,f):
    pass

Once, $J$ and $r$ are computed we can solve the system $$ (J^T J + \lambda \mathbb{I}) \delta_x =  - J^T\ r,$$
since we are using an iterative solver, we need to define an initial guess $X_0$ that should have the same dimension as the columns of $J$


In [13]:
# X_0

n_f = np.ones((3*F,3)).flatten()

X_0 = np.concatenate((v.flatten(), n_f))

In [ ]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, linalg

 # Compute pseudo Hessian
H = J.T@J
H[np.diag_indices_from(H)] += np.diag(H).max()*1e-6

# Sparse matrix H
H = csc_matrix(H)

# Solve for dx
dx = linalg.spsolve(H, -J.T@r)

# Update vertices
X = X_0 + dx

# energy
energy = r^t@r

This just represent one step of the optmization. Try to visualize the results and create a function that run the optimization n times. 